![example](images/director_shot.jpeg)

# Major Motion Picture Box Office Analysis 

**Authors:** Billy Lopez, Elliot Macy
***

## Overview

This project analyzes 20 years of movie data and offers strategic insights for Microsoft to launch a financially sucessful film studio.

## Business Problem

We answer three questions relating to box office profit and revenue for major motion pictures from the past 20 years.

1. How does the correlation between runtime and revenue vary between different MPAA ratings?
2. Which months of the year and days of the week are the optimal releases dates?
3. Do the optimal release dates differ from foreign to domestic audiences?

Descriptive analysis of the relationshipes between profit and revenue, on the one hand, and release dates and runtimes, on the other, points toward very different approaches depending on MPAA rating and audience.

Namely, revenues increase with strciter MPAA ratings from G to PG-13 but fall dramtically for R and NC-17 ratings. Optimizing runtimes for maximum revenue depends on the target rating: longer movies do better with stricter ratings.

Moreover, profits peak yearly for movies released in June and December and peak weekly for Tuesday and Wednesday releases. However, upon further analysis, we found that movies intended for international audiences peak monthly in July only and weekly on Mondays.

## Data Understanding

Our analysis utilizes data obtained from Rotten Tomatoes (RT) and The Movie Database (TMDB). The combined dataset represents the profit, revenue, and other key features of all 21st century major motion pictures. We focus on release date, audience, rating, and runtime, and how these features relate with profit and revenue as well as each other.

In [ ]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
import matplotlib.ticker as ticker
import seaborn as sns
import requests
import json
import time
import random
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
from datetime import datetime as dt
register_matplotlib_converters()
from bs4 import BeautifulSoup

In [ ]:
# Explore data
rotn = pd.read_csv('data/rotn.csv')
tmdb_raw = pd.read_csv('data/tmdb.csv', index_col=0, encoding='Latin-1')
tmdb = pd.DataFrame.from_dict(tmdb_raw)
pd.set_option('display.max_colwidth', -1, 'display.float_format', lambda x: '%.5f' % x)

## Data Preparation

To prepare our data for analysis we dropped the unrelated columns and engineered several new features. After selecting our features, we dropped movies missing quantitative values. For missing categorical values, we either dropped the movie in question or replaced the value with 'None'.

In [ ]:
# ————> Clean RT Data: <————
    
# Remove ':' from column names
rotn.columns = [col[:-1] for col in rotn.columns]

# Drop extraneous columns
rotn = rotn.loc[:,['Title', 'Box Office (Gross USA)', 'Runtime', 'Rating']]

# Drop rows with null values
rotn.dropna(inplace=True)

# Replace null values with 'None'
rotn.fillna('None', inplace=True)

# Reset index
rotn.reset_index(inplace=True, drop=True)

# Remove components from Ratings
rotn['Rating'] = rotn['Rating'].map(lambda x : x.split(' ')[0])

#Remove '$' and 'M' and convert Gross to int
rotn['Box Office (Gross USA)'] = rotn['Box Office (Gross USA)'].map(
    lambda x : int(float(x.strip('$M')) * 1000000) if x.endswith('M')
    else int(float(x.strip('$K')) * 1000))

# Convert Runtime to minutes
rotn['Runtime'] = rotn['Runtime'].map(
    lambda x : int(x[0])*60 + int(x[-3:-1]) if 'h' in x else int(x[:2]))

In [ ]:
# ————> Clean TMDB Data: <————

# Drop 2 movies with status = 'Rumored'or'Post Production'
tmdb.drop(tmdb.query('status != "Released"').index[0], inplace=True)
tmdb.drop(tmdb.query('status != "Released"').index[0], inplace=True)

# Add profit and audience columns with 'revenue' - 'budget' and 'original_language' == 'en'
tmdb['profit'] = tmdb.revenue - tmdb.budget
tmdb['audience'] = tmdb['original_language'].map(
    lambda x : 'Domestic' if x == 'en' else 'Foreign')# <---Adding new column *now* in order

# Drop extraneous columns
tmdb = tmdb.loc[:,['title', 'revenue', 'budget', 'profit', 'release_date', 'audience']]

# Drop rows with null values
tmdb.dropna(inplace=True)

# Drop movies from before 2000 or after 2019 with budgets < 1m
tmdb = tmdb[(tmdb.release_date.between('2000-01-01', '2019-01-01')) & (tmdb.budget > 1000000)]

# Reset index
tmdb.reset_index(inplace=True, drop=True)

# Convert profit to int after dropping null values
tmdb['profit'] = tmdb.profit.map(lambda x : int(x))

# Convert floats to ints
tmdb.budget = tmdb.budget.map(lambda x : int(x))
tmdb.revenue = tmdb.revenue.map(lambda x : int(x))

# Convert strs to datetime objs
tmdb.release_date = tmdb.release_date.map(lambda x : dt.strptime(x.split('\n')[0], '%Y-%m-%d'))

## Data Modeling

We began by modeling the relationship between revenue and runtime as well as profit and release date. Finding these results insightful, we expanded our analysis to include additional features: runtime, in relation to revenue and rating, and audience, in relation to profit and release date.

In [ ]:
# ————> Modeling Major Motion Picture Profit by Release Month (Foreign & Domestic): <————

# Model reveneue by rating
r_and_r_bar, ax = plt.subplots(figsize = (8,10));
x = rotn.groupby(['Rating'])['Box Office (Gross USA)'].mean().index
y = rotn.groupby(['Rating'])['Box Office (Gross USA)'].mean()
ax.set_title('Ratings vs Revenue', fontdict = {'size': 20, 'weight':'bold'})
ax.set_xlabel('Ratings', fontdict= {'size':15})
ax.set_ylabel('Revenue', fontdict= {'size':15});
ax.text(-.03, -.1, "Data source: rottentomatoes.com", 
        fontsize=10, horizontalalignment='center', 
        verticalalignment='center', transform=ax.transAxes)
ax.bar(x,y)

# Model revenue by rating and runtime
rotn['Gross in Millions'] = rotn['Box Office (Gross USA)'] / 1000000
fig, ax = plt.subplots(figsize = (8,10))
sns.scatterplot(data = rotn, x = 'Runtime', y='Gross in Millions', 
               hue = 'Rating', ax=ax)
ax.set_title('Correlation of Profits by Runtime and Ratings', fontdict={'size':18, 'weight':'bold'})
ax.set_ylabel('Box Office (Gross USA) in millions')
ax.text(-.03, -.1, "Data source: rottentomatoes.com", 
        fontsize=10, horizontalalignment='center', 
        verticalalignment='center', transform=ax.transAxes)
plt.show()

In [ ]:
# ————> Modeling Major Motion Picture Profit by Release Month & Day: <————

# Model by release month
fig, ax = plt.subplots(figsize=(10,7.5), facecolor='white')

x = ['Jan.', 'Feb.', 'Mar.', 'Apr.', 'May', 'Jun.', 'Jul.', 'Aug.', 'Sep.', 'Oct.', 'Nov.', 'Dec.']
y = tmdb.groupby(tmdb['release_date'].dt.month).profit.mean().values

plt.style.use('seaborn')
ax.set_title('Profit by Release Month', fontsize=22)
ax.title.set_position([.5, 1.03])

plt.ticklabel_format(style='plain')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
formatter = ticker.FormatStrFormatter('$%1.0f')
ax.yaxis.set_major_formatter(formatter)

plt.text(-.03, -.1, "Data source: tmdb.com", fontsize=10, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

ax.plot(x,y)
plt.show()

# Model by release day
fig, ax = plt.subplots(figsize=(10,7.5), facecolor='white')
data = tmdb.groupby(tmdb['release_date'].dt.weekday_name).profit.mean().reindex(
    ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'])

x = data.index
y = data.values

plt.style.use('seaborn')
ax.set_title('Profit by Release Day', fontsize=22)
ax.title.set_position([.5, 1.03])

plt.ticklabel_format(style='plain')
plt.xticks(fontsize=14)    
plt.yticks(fontsize=14)    
formatter = ticker.FormatStrFormatter('$%1.0f')
ax.yaxis.set_major_formatter(formatter)

plt.text(-.03, -.1, "Data source: tmdb.com", fontsize=10, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

ax.plot(x,y)
plt.show()

In [ ]:
# ————> Modeling Major Motion Picture Profit by Release Month & Day (Foreign x Domestic): <————

# Model by release month
fig, ax = plt.subplots(figsize=(10,7.5), facecolor='white')
plt.style.use('seaborn')
plt.ticklabel_format(style='plain')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
formatter = ticker.FormatStrFormatter('$%1.0f')
ax.yaxis.set_major_formatter(formatter)

x = ['Jan.', 'Feb.', 'Mar.', 'Apr.', 'May', 'Jun.', 'Jul.', 'Aug.', 'Sep.', 'Oct.', 'Nov.', 'Dec.']
y = tmdb[tmdb['audience'] == 'Foreign'].groupby(tmdb['release_date'].dt.month).profit.mean().values
ax.set_title('Profit by Release Month', fontsize=22)
ax.plot(x,y)

x = ['Jan.', 'Feb.', 'Mar.', 'Apr.', 'May', 'Jun.', 'Jul.', 'Aug.', 'Sep.', 'Oct.', 'Nov.', 'Dec.']
y = tmdb[tmdb['audience'] == 'Domestic'].groupby(tmdb['release_date'].dt.month).profit.mean().values
ax.set_title('Major Motion Picture Profit by Release Month (Foreign & Domestic)', fontsize=22)
ax.title.set_position([.429, 1.03])
ax.plot(x,y)

plt.text(-.03, -.1, "Data source: tmdb.com", fontsize=10, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
plt.legend(['Foreign', 'Domestic'])
plt.show()

# Model by release day
fig, ax = plt.subplots(figsize=(10,7.5), facecolor='white')
plt.style.use('seaborn')
plt.ticklabel_format(style='plain')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
formatter = ticker.FormatStrFormatter('$%1.0f')
ax.yaxis.set_major_formatter(formatter)

data = tmdb[tmdb['audience'] == 'Foreign'].groupby(tmdb['release_date'].dt.weekday_name).profit.mean().reindex(
    ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'])
x = data.index
y = data.values
ax.set_title('Profit by Release Month', fontsize=22)
ax.plot(x,y)

data = tmdb[tmdb['audience'] == 'Domestic'].groupby(tmdb['release_date'].dt.weekday_name).profit.mean().reindex(
    ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'])
x = data.index
y = data.values
ax.set_title('Major Motion Picture Profit by Release Day (Foreign & Domestic)', fontsize=22)
ax.title.set_position([.42, 1.03])
ax.plot(x,y)

plt.text(-.03, -.1, "Data source: tmdb.com", fontsize=10, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
plt.legend(['Foreign', 'Domestic'])
plt.show()

## Conclusion
As demonstrated, revenues increase with strciter MPAA ratings from G to PG-13 but fall dramtically for R and NC-17 ratings. Optimizing runtimes for maximum revenue depends on the target rating: longer movies do better with stricter ratings.

Moreover, profits peak yearly for movies released in June and December and peak weekly for Tuesday and Wednesday releases. However, upon further analysis, we found that movies intended for international audiences peak monthly in July only and weekly on Mondays.

Our analysis leads to several reccomendations for Microsoft to maximize their film studio's profit and revenue.
1. Look for projects likely to receive pg-13 ratings while ensuring they are not rated R.
2. Restrict G and PG rated movies to shorter durations than movies rated PG-13 and R.
3. Plan domestic release dates for Tuesdays and Wednesdays in June and January. For interntational markets, plan releases for Mondays in July.